In [1]:
#%matplotlib notebook
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from neutral_surfaces import pot_dens_surf
#from neutral_surfaces._densjmd95 import rho_ufunc

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No conversion from UniTuple(float64 x 2) to float64 for '$46return_value.5', defined at None

File "../../../neutral_surfaces/interp_ppc.py", line 44:
def val(X, Y, Yppc, x):
    <source elided>
    if np.isnan(x) or x < X[0] or X[-1] < x:
        return (np.nan, np.nan)
        ^

During: typing of assignment at /home/stanley/Dropbox/work/projects-gfd/neutral-surfaces-python/neutral_surfaces/interp_ppc.py (44)

File "../../../neutral_surfaces/interp_ppc.py", line 44:
def val(X, Y, Yppc, x):
    <source elided>
    if np.isnan(x) or x < X[0] or X[-1] < x:
        return (np.nan, np.nan)
        ^


In [ ]:
OCCA_dir = "~/work/data/OCCA/"
xrs = xr.open_dataset(OCCA_dir + "DDsalt.0406annclim.nc")
xrt = xr.open_dataset(OCCA_dir + "DDtheta.0406annclim.nc")
lon = xrs.Longitude_t.values
lat = xrs.Latitude_t.values
S = np.require(xrs.salt.values[0] , dtype=np.float64, requirements="C")  # pick first time
T = np.require(xrt.theta.values[0], dtype=np.float64, requirements="C")  # pick first time
Z = np.require(xrs.Depth_c, dtype=np.float64, requirements="C")     # DEV: currently ignoring distinction between Z and P, until Boussinesq equation of state is ready. 
S = np.moveaxis(S, 0, -1) # Move vertical axis to end
T = np.moveaxis(T, 0, -1) # Move vertical axis to end

In [ ]:
# Here we discard the unneeded time axis.
%time s, t, p = pot_dens_surf(S, T, Z, 0., 1026., axis=-1, tol=1e-4)
# 202 ms ± 3.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)  when not overwriting p in vertsolve
# 202 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)   when overwriting p in vertsolve

In [ ]:
fig, ax = plt.subplots()
cs = ax.contourf(lon, lat, p)
cbar = fig.colorbar(cs, ax=ax)
cbar.set_label("Pressure [dbar]")
ax.set_title(r"Depth of $\sigma_\theta = 26$ in OCCA")

In [ ]:
# %%timeit
# s, t, p = pot_dens_surf(xrst.salt.values, 
#                         xrst.theta.values, 
#                         xrst.p.values, 
#                         0., 1026., axis=1, tol=1e-4)